In [1]:
import pandas as pd
import os


datasets_list = os.listdir('Generated_Data/DataFrame/')


#datasets_list = ['combined_binary_en_HateCheck.csv','combined_binary_en_yt_reddit.csv', 'EN_combined_binary.csv',]

In [7]:
ind = 4
df = pd.read_csv(f'Datasets/{datasets_list[ind]}')
df = df.dropna(subset=['text','binary_labels'])
df = df.reset_index(drop=True,inplace=False)

df.head()

,text,labels,binary_labels
0,The reason you see Indians bashing Muslims and...,1.0,1
1,Why don't they have their own games?,0.0,0
2,fuck bulgaria stupid country,1.0,1
3,Finally someone who means what he says. It's ...,0.0,0
4,Who do you think you are cunt? wtf! Stay stead...,1.0,1


In [2]:
df_classifiers = {}
classifiers = ['HateCheck', 'stormfront', 'twitter', 'fb_yt', 'convAI','wikipedia']


In [3]:
from transformers import pipeline

for model in classifiers:
    df_classifiers[model] = pipeline('text-classification', model=f'Output_Dir/{model}', device=0)

In [5]:
from transformers import pipeline

model = pipeline('text-classification', model=f'Output_Dir/{classifiers[4]}')

In [6]:
model(['Hello','Hi'])

[{'label': 0, 'score': 0.9990739822387695},
 {'label': 0, 'score': 0.9988235831260681}]

In [ ]:
import pandas as pd
from tqdm import tqdm


cnt = 4
for dataset in tqdm(datasets_list):
    if(not dataset.endswith('.csv')):
        continue
    df_data = pd.read_csv(f'Generated_Data/DataFrame/{dataset}')
    
    df_new = {}
    for col in df_data.columns:
        df_new[col] = []
    df_new['toxic_count'] = []
    df_new['non_toxic_count'] = []
    for c in classifiers:
        df_new[c] = []
    
    toxic_list = []
    non_toxic_list = []
    for ind in range(len(df_data['data_id'])):
        if(df_data['source_label'][ind]=='toxic'):
            toxic_list.append(df_data['source_text'][ind])
            non_toxic_list.append(df_data['target_text'][ind])
        else:
            non_toxic_list.append(df_data['source_text'][ind])
            toxic_list.append(df_data['target_text'][ind])
        

        for col in df_data.columns:
            df_new[col].append(df_data[col][ind])

    tmp_data = {}


    for model in classifiers:
        tmp_data[f'toxic_{model}'] = df_classifiers[model](toxic_list, max_length = 512, 
                                                           padding = 'max_length', truncation=True)
        tmp_data[f'non_toxic_{model}'] = df_classifiers[model](non_toxic_list, max_length = 512,
                                                               padding = 'max_length', truncation=True)

    for k in range(len(df_data['source_text'])): #zip(tmp_data[f'toxic_{model}'],tmp_data[f'non_toxic_{model}']):
        tox_cnt = non_tox_cnt = 0
        for model in classifiers:
            tox = tmp_data[f'toxic_{model}'][k]['label']
            non_tox = tmp_data[f'non_toxic_{model}'][k]['label']
            df_new[model].append(f'{tox}_{non_tox}')
            tox_cnt+= int(tox)
            non_tox_cnt += int(non_tox)

        df_new['toxic_count'].append(tox_cnt)
        df_new['non_toxic_count'].append(non_tox_cnt)   

    

    df_new = pd.DataFrame(df_new)
    df_new.to_csv(f'Generated_Data/Pred_DataFrame/{dataset}',index=False)
        
        #print(f'Dataset: {dataset} ## Original Length:{len(df_data['source_text'])} ## New ')


In [ ]:
!./run_python_refine_1.sh

In [4]:
import pandas as pd
import os

file_list = os.listdir('Generated_Data/Pred_3K_DataFrame')

total_ori = 0
total_new = 0

for file in file_list:
    if(not file.endswith('.csv')):
        continue
    df_ori = pd.read_csv(f'Generated_Data/Pred_3K_DataFrame/{file}')
    df_new = df_ori[(df_ori['toxic_count']>0) & (df_ori['non_toxic_count']<1)]
    
    print(f"{file}: Original: {len(df_ori['source_text'])} ## Modified: {len(df_new['source_text'])}")
    total_ori += len(df_ori['source_text'])
    total_new += len(df_new['source_text'])
    
print(f"TOTAL Ori: {total_ori} ### TOTAL Modified: {total_new}")

hatecheck_3000.csv: Original: 2741 ## Modified: 1398
twitter and facebook_3000.csv: Original: 3000 ## Modified: 1749
wikipedia_3000.csv: Original: 2995 ## Modified: 2153
twitter_3000.csv: Original: 3000 ## Modified: 2337
convai_3000.csv: Original: 650 ## Modified: 552
fb_yt_3000.csv: Original: 2897 ## Modified: 1901
yt_reddit_3000.csv: Original: 222 ## Modified: 156
fox news_3000.csv: Original: 1104 ## Modified: 831
stormfront_3000.csv: Original: 3000 ## Modified: 2511
gab_3000.csv: Original: 3000 ## Modified: 2151
reddit_3000.csv: Original: 3000 ## Modified: 2222
TOTAL Ori: 25609 ### TOTAL Modified: 17961


In [ ]:
#CREATE DATASET FOR FINETUNING

In [7]:
import pandas as pd
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

file_list = os.listdir('Datasets/Coverted_Col_Name_3K')

df_train_all = pd.DataFrame()
df_val_all = pd.DataFrame()
df_test_all = pd.DataFrame()

for file in tqdm(file_list):
    if(not file.endswith('.csv')):
        continue
    df_ori = pd.read_csv(f'Datasets/Coverted_Col_Name_3K/{file}')
    df_new = df_ori[(df_ori['toxic_count']>0) & (df_ori['non_toxic_count']<1)]
    
    TEST_SIZE = 100
    if(file.startswith('yt_reddit')):
        TEST_SIZE = 50
    
    df_train_val, df_test = train_test_split(df_new, test_size=TEST_SIZE)
    df_train, df_val = train_test_split(df_train_val, test_size=TEST_SIZE)
    
    df_train_all = pd.concat([df_train_all,df_train], ignore_index=True)
    df_val_all = pd.concat([df_val_all,df_val], ignore_index=True)
    df_test_all = pd.concat([df_test_all,df_test], ignore_index=True)


for i in range(5):
    df_train_all = shuffle(df_train_all)
    df_train_all = df_train_all.reset_index(inplace=False, drop=True)
    df_val_all = shuffle(df_val_all)
    df_val_all = df_val_all.reset_index(inplace=False, drop=True)
    df_test_all = shuffle(df_test_all)
    df_test_all = df_test_all.reset_index(inplace=False, drop=True)


#print(f"{file}: Original: {len(df_ori['source_text'])} ## Modified: {len(df_new['source_text'])}")
df_train_all.to_csv('Datasets/Toxicity_All_Platform_3K/train/train_All_Platform_3K.csv', index=False)
df_val_all.to_csv('Datasets/Toxicity_All_Platform_3K/validation/val_All_Platform_3K.csv', index=False)
df_test_all.to_csv('Datasets/Toxicity_All_Platform_3K/test/test_All_Platform_3K.csv', index=False)

100%|██████████| 11/11 [00:00<00:00, 20.53it/s]


In [2]:
import pandas as pd
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

file_list = os.listdir('Datasets/Coverted_Col_Name_3K')

df_train_all = pd.DataFrame()
df_val_all = pd.DataFrame()
df_test_all = pd.read_csv('Datasets/Toxicity_All_Platform_3K/test/test_All_Platform_3K.csv')


for file in tqdm(file_list):
    if(not file.endswith('.csv')):
        continue
    df_ori = pd.read_csv(f'Datasets/Coverted_Col_Name_3K/{file}')
    df_new = df_ori[(df_ori['toxic_count']>0) & (df_ori['non_toxic_count']<1)]
    print(len(df_new['data_id']))
    id_list = df_test_all['data_id'].tolist()
    df_new = df_new[df_new['data_id'].isin(id_list)==False]
    print(len(df_new['data_id']))
    
    
    TEST_SIZE = 100
    if(file.startswith('yt_reddit')):
        TEST_SIZE = 50
    
    df_train, df_val = train_test_split(df_new, test_size=TEST_SIZE)
    #df_train, df_val = train_test_split(df_train_val, test_size=TEST_SIZE)
    
    df_train_all = pd.concat([df_train_all,df_train], ignore_index=True)
    df_val_all = pd.concat([df_val_all,df_val], ignore_index=True)
    #df_test_all = pd.concat([df_test_all,df_test], ignore_index=True)


for i in range(5):
    df_train_all = shuffle(df_train_all)
    df_train_all = df_train_all.reset_index(inplace=False, drop=True)
    df_val_all = shuffle(df_val_all)
    df_val_all = df_val_all.reset_index(inplace=False, drop=True)
    #df_test_all = shuffle(df_test_all)
    #df_test_all = df_test_all.reset_index(inplace=False, drop=True)


#print(f"{file}: Original: {len(df_ori['source_text'])} ## Modified: {len(df_new['source_text'])}")
df_train_all.to_csv('Datasets/Toxicity_All_Platform_3K/train/train_All_Platform_3K.csv', index=False)
df_val_all.to_csv('Datasets/Toxicity_All_Platform_3K/validation/val_All_Platform_3K.csv', index=False)
#df_test_all.to_csv('Datasets/Toxicity_All_Platform_3K/test/test_All_Platform_3K.csv', index=False)

 25%|██▌       | 3/12 [00:00<00:00, 15.45it/s]

1398
1298
1749
1649


 42%|████▏     | 5/12 [00:00<00:00,  7.75it/s]

2153
2053
2337
2237


 58%|█████▊    | 7/12 [00:00<00:00,  7.91it/s]

...

156
106
831
731


 92%|█████████▏| 11/12 [00:01<00:00,  7.84it/s]

2511
2411
2151
2051


100%|██████████| 12/12 [00:01<00:00,  7.67it/s]

2222
2122


In [ ]:
#CREATE DATASET FOR PLATFORM-SPECIFIC FINETUNING

In [11]:
import pandas as pd
import os
from tqdm import tqdm

#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

SPLIT = {
    'train': ['wikipedia', 'reddit', 'twitter and facebook', 'twitter'],
    'validation': ['yt_reddit','fb_yt'],
    'test': ['convai', 'fox news', 'gab', 'hatecheck', 'stormfront'],
}

file_list = os.listdir('Datasets/Coverted_Col_Name_3K')

df_train_all = pd.DataFrame()
df_val_all = pd.DataFrame()
df_test_all = pd.DataFrame()

#print(SPLIT['train'])

for file in tqdm(file_list):
    #print(file.split('_1000')[0])
    if(not file.endswith('.csv')):
        continue
    df_ori = pd.read_csv(f'Datasets/Coverted_Col_Name_3K/{file}')
    df_new = df_ori[(df_ori['toxic_count']>0) & (df_ori['non_toxic_count']<1)]
    
    
    if(file.split('_3000')[0] in SPLIT['train']):
        df_train_all = pd.concat([df_train_all,df_new], ignore_index=True)
    elif(file.split('_3000')[0] in SPLIT['validation']):
        df_val_all = pd.concat([df_val_all,df_new], ignore_index=True)
    elif(file.split('_3000')[0] in SPLIT['test']):
        df_test_all = pd.concat([df_test_all,df_new], ignore_index=True)


for i in range(5):
    df_train_all = shuffle(df_train_all)
    df_train_all = df_train_all.reset_index(inplace=False, drop=True)
    df_val_all = shuffle(df_val_all)
    df_val_all = df_val_all.reset_index(inplace=False, drop=True)
    df_test_all = shuffle(df_test_all)
    df_test_all = df_test_all.reset_index(inplace=False, drop=True)


#print(f"{file}: Original: {len(df_ori['source_text'])} ## Modified: {len(df_new['source_text'])}")
df_train_all.to_csv('Datasets/Toxicity_Specific_Platform_3K/train/train_Specific_Platform_3K.csv', index=False)
df_val_all.to_csv('Datasets/Toxicity_Specific_Platform_3K/validation/val_Specific_Platform_3K.csv', index=False)
df_test_all.to_csv('Datasets/Toxicity_Specific_Platform_3K/test/test_Specific_Platform_3K.csv', index=False)

100%|██████████| 12/12 [00:00<00:00, 27.26it/s]


In [16]:
from evaluate import load
bertscore = load("bertscore")

Matplotlib is building the font cache; this may take a moment.


In [28]:
model_type = "princeton-nlp/sup-simcse-roberta-large" #"distilbert-base-uncased"

predictions = ["All women should die"]
references = ["No men should die"]

results = bertscore.compute(predictions=predictions, references=references, model_type=model_type, device=0)
print(results)

{'precision': [0.9530810117721558], 'recall': [0.9530810117721558], 'f1': [0.9530810117721558], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.28.1)'}


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [18]:

sentences = ["All women should die", 
             "No women should die"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)
## tensor([[0.6003]])


tensor([[0.8734]], device='cuda:0')